In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib

In [51]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

In [52]:
#Use `koi_disposition` for the y values
X=df[['koi_steff_err1','koi_model_snr', 'koi_duration_err2', 'koi_steff_err2', 'koi_duration_err1', 'koi_prad', 'koi_fpflag_co',
    'koi_prad_err1', 'koi_time0bk_err1', 'koi_time0bk_err2' ]]
feature_names = X.columns
y=df["koi_disposition"]

def functionrand(x):
    if x == 'CONFIRMED':
        return 1
    else:
        return 0
y = y.apply(functionrand)
y[:5]

0    1
1    0
2    0
3    1
4    1
Name: koi_disposition, dtype: int64

In [53]:
print(X.shape, y.shape)

(6991, 10) (6991,)


In [54]:
# Split data into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [55]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [56]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [57]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression()

In [58]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.788670608430288
Testing Data Score: 0.7631578947368421


In [59]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
grid = GridSearchCV(classifier, param_grid, verbose=3)

In [60]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 490 candidates, totalling 2450 fits
[CV] C=0.001, max_iter=100, penalty=l1, solver=newton-cg .............
[CV]  C=0.001, max_iter=100, penalty=l1, solver=newton-cg, score=nan, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l1, solver=newton-cg .............
[CV]  C=0.001, max_iter=100, penalty=l1, solver=newton-cg, score=nan, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l1, solver=newton-cg .............
[CV]  C=0.001, max_iter=100, penalty=l1, solver=newton-cg, score=nan, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l1, solver=newton-cg .............
[CV]  C=0.001, max_iter=100, penalty=l1, solver=newton-cg, score=nan, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l1, solver=newton-cg .............
[CV]  C=0.001, max_iter=100, penalty=l1, solver=newton-cg, score=nan, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l1, solver=lbfgs .................
[CV]  C=0.001, max_iter=100, penalty=l1, solver=lbfgs, score=nan, total=   0.0s
[CV] C=0.0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\hanie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on t


[CV] C=0.001, max_iter=100, penalty=l2, solver=liblinear .............
[CV]  C=0.001, max_iter=100, penalty=l2, solver=liblinear, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=liblinear .............
[CV]  C=0.001, max_iter=100, penalty=l2, solver=liblinear, score=0.748, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=liblinear .............
[CV]  C=0.001, max_iter=100, penalty=l2, solver=liblinear, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=liblinear .............
[CV]  C=0.001, max_iter=100, penalty=l2, solver=liblinear, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=100, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=100, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=100, penalty=l2, solver=sag ..................

C:\Users\hanie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\hanie\anaconda3\lib\site-packages\sklearn\model_selectio


[CV] C=0.001, max_iter=200, penalty=l2, solver=liblinear .............
[CV]  C=0.001, max_iter=200, penalty=l2, solver=liblinear, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=200, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=200, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=200, penalty=l2, solver=sag, score=0.748, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=200, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=sag ...................
[CV]  C=0.001, max_iter=200, penalty=l2, solver=sag, score=0.749, total=   0.0s
[CV] C=0.001, max_iter=200, penalty=l2, solver=saga ..................
[CV]  C=0.001, m

...

C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV]  C=1000, max_iter=700, penalty=l2, solver=saga, score=0.831, total=   0.5s
[CV] C=1000, max_iter=700, penalty=l2, solver=saga ...................


C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done 2450 out of 2450 | elapsed:  3.3min finished


[CV]  C=1000, max_iter=700, penalty=l2, solver=saga, score=0.823, total=   0.5s


GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [100, 200, 300, 400, 500, 600, 700],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=3)

In [61]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1000, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
0.8586623756540217


In [68]:
classifier_final=LogisticRegression(C=1.0, max_iter= 200, penalty= 'l1', solver='saga')
classifier_final.fit(X_train_scaled, y_train)

C:\Users\hanie\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(max_iter=200, penalty='l1', solver='saga')

In [69]:
classifier_final.score(X_test_scaled, y_test)

0.8094965675057209

In [70]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'logisticregression.sav'
joblib.dump(classifier_final, filename)

['logisticregression.sav']